In [1]:
pip install pytorch_tabular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 10.9 MB/s eta 0:00:00

In [6]:
import numpy as np
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib

from utils import calculate_metric

from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig
)
from pytorch_tabular import model_sweep
import joblib

In [7]:
# set random seeds
np.random.seed(0)
torch.manual_seed(0)

In [9]:
INPUT_FILE = "../data/data_removing_na.xlsx"
BATCH_SIZE = 32
PROJECT_NAME = "run/all"

DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
DEVICE_LIST = [3, 2, 1, 0]

In [10]:
DEVICE

device(type='cuda', index=1)

In [11]:
df = pd.read_excel(INPUT_FILE)

In [12]:
df.head()

,rr1_30,currency,seniorioty_adj,coupon rate,domicile_country,exchange_country,Industry_sector,Industry_group,Industry_subgroup,event_type,...,PD_55_pd,PD_56_pd,PD_57_pd,PD_58_pd,PD_59_pd,PD_60_pd,DTD,NI_Over_TA,Size,defaulted_in_last_6_months
0,0.259908,USD,Senior Subordinated Unsecured,9.000,United States,United States,Consumer Discretionary,Retail & Whsle - Discretionary,E-Commerce Discretionary,Bankruptcy Filing,...,0.396731,0.397453,0.398148,0.398819,0.399467,0.400092,-0.732815,-0.007137,-0.852484,False
1,0.032729,USD,Senior Subordinated Unsecured,5.750,United States,United States,Health Care,Health Care,Health Care Facilities & Svcs,Default Corp Action,...,0.957454,0.957467,0.957480,0.957492,0.957503,0.957514,-1.666262,-0.000286,-1.186347,False
2,0.972400,USD,Unsecured,5.675,South Korea,South Korea,Consumer Discretionary,Retail & Whsle - Discretionary,Wholesale - Discretionary,Default Corp Action,...,0.568169,0.568693,0.569197,0.569682,0.570150,0.570600,-1.853366,0.000191,1.053677,False
3,1.047416,CHF,Unsecured,0.125,South Korea,South Korea,Consumer Discretionary,Retail & Whsle - Discretionary,Wholesale - Discretionary,Default Corp Action,...,0.568169,0.568693,0.569197,0.569682,0.570150,0.570600,-1.853366,0.000191,1.053677,False
4,0.848872,JPY,Unsecured,1.750,Japan,Japan,Industrials,Industrial Products,Electrical Equipment,Bankruptcy Filing,...,0.130285,0.130688,0.131081,0.131465,0.131840,0.132206,-0.768857,-0.028058,-1.946507,False


In [13]:
df.shape

(1725, 165)

In [14]:
feature_list = df.columns
feature_list = feature_list.drop('rr1_30')

In [15]:
feature_list

Index(['currency', 'seniorioty_adj', 'coupon rate', 'domicile_country',
       'exchange_country', 'Industry_sector', 'Industry_group',
       'Industry_subgroup', 'event_type', 'event_type_subcategory_sum',
       ...
       'PD_55_pd', 'PD_56_pd', 'PD_57_pd', 'PD_58_pd', 'PD_59_pd', 'PD_60_pd',
       'DTD', 'NI_Over_TA', 'Size', 'defaulted_in_last_6_months'],
      dtype='object', length=164)

In [16]:
category_features = list(df.select_dtypes(include=['object', 'bool']).columns)
non_category_features = [i for i in feature_list if i not in category_features]

In [17]:
print(len(non_category_features))
print(len(category_features))

153
11


In [18]:
category_features

['currency',
 'seniorioty_adj',
 'domicile_country',
 'exchange_country',
 'Industry_sector',
 'Industry_group',
 'Industry_subgroup',
 'event_type',
 'event_type_subcategory_sum',
 'defaulted_in_last_5_years',
 'defaulted_in_last_6_months']

In [19]:
# split data into training and test set
test_size = 0.25
train, test = train_test_split(df, test_size=test_size, random_state=42)
train, valid = train_test_split(train, test_size=test_size, random_state=42)

In [20]:
train.head()

,rr1_30,currency,seniorioty_adj,coupon rate,domicile_country,exchange_country,Industry_sector,Industry_group,Industry_subgroup,event_type,...,PD_55_pd,PD_56_pd,PD_57_pd,PD_58_pd,PD_59_pd,PD_60_pd,DTD,NI_Over_TA,Size,defaulted_in_last_6_months
536,0.182132,USD,Senior Subordinated Unsecured,2.750,United States,United States,Technology,Tech Hardware & Semiconductors,Technology Hardware,Bankruptcy Filing,...,0.563420,0.563890,0.564342,0.564777,0.565194,0.565597,-1.004436,-0.082341,-3.870640,False
765,0.056740,USD,Senior Unsecured,8.125,United States,United States,Energy,Oil & Gas,Oil & Gas Producers,Default Corp Action,...,0.691011,0.691382,0.691738,0.692080,0.692408,0.692725,-1.084433,-0.052027,-2.074964,False
222,0.603846,USD,Senior Unsecured,8.625,United States,United States,Industrials,Industrial Services,Transportation & Logistics,Bankruptcy Filing,...,0.585977,0.586288,0.586585,0.586867,0.587137,0.587395,-1.115711,-0.007636,-4.029905,False
555,0.836365,USD,Senior Unsecured,9.500,India,India,Utilities,Utilities,Electric Utilities,Default Corp Action,...,0.462222,0.467771,0.473250,0.478659,0.484000,0.489275,-0.751984,-0.000207,3.582827,False
1527,0.459547,CNY,Senior Unsecured,6.600,China,China,Real Estate,Real Estate,Real Estate Owners & Developers,Default Corp Action,...,0.253968,0.256798,0.259601,0.262379,0.265132,0.267862,-0.427533,0.002135,0.722957,False


RUNNING THE MODEL

In [21]:
# empty cache first
torch.cuda.empty_cache()

In [23]:
EPOCHS = 100
target_col = 'rr1_30'

data_config = DataConfig(
    target=[
        target_col
    ],  # target should always be a list
    continuous_cols=non_category_features,
    categorical_cols=category_features,
)
trainer_config = TrainerConfig(
    batch_size=BATCH_SIZE,
    max_epochs=EPOCHS,
    early_stopping=None,
    accelerator="gpu",
    load_best=True,
    devices_list=DEVICE_LIST
)
optimizer_config = OptimizerConfig()
experiment_config = ExperimentConfig(project_name=PROJECT_NAME,exp_watch="all",log_target="tensorboard")

In [24]:
EPOCHS = 100

trainer_config = TrainerConfig(
    batch_size=BATCH_SIZE,
    max_epochs=EPOCHS,
    early_stopping=None
)


sweep_df, best_model = model_sweep(
                            task="regression",
                            train=train,
                            test=valid,
                            data_config=data_config,
                            optimizer_config=optimizer_config,
                            trainer_config=trainer_config,
                            model_list="high_memory",
                            verbose=True # Make True if you want to log metrics and params each trial
                        )


Output()

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_tabular/tabular_model_sweep.py:261: UserWarning: Training Progress bar is not `none`. Set `progress_bar=none` in `trainer_config` to remove this warning
  warnings.warn(
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_tabular/tabular_model_sweep.py:268: UserWarning: The full model list is quite large and uses a lot of memory. Consider using `lite` or define configs yourselves for a faster run
  warnings.warn(
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.m

2025-03-24 03:09:25,251 - {pytorch_tabular:328} - INFO - Training AutoIntModel

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:
654: Checkpoint directory /home/nguyenhoa/hoa/up5/final/src/saved_models exists and is not empty.

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connec
tor.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the 
value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.

...

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connec
tor.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the 
value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.

2025-03-24 03:46:51,986 - {pytorch_tabular:382} - INFO - Finished Training TabTransformerModel

2025-03-24 03:46:51,989 - {pytorch_tabular:383} - INFO - Results: model: TabTransformerModel, # Params: 277 T,     
epochs: 100, test_loss: 0.06665383279323578, test_mean_squared_error: 0.06665383279323578, time_taken:             
105.43194341659546, time_taken_per_epoch: 1.0543194341659545

2025-03-24 03:46:51,998 - {pytorch_tabular:388} - INFO - Model Sweep Finished

2025-03-24 03:46:52,000 - {pytorch_tabular:389} - INFO - Best Model: GANDALFModel

In [25]:
sweep_df

,model,# Params,epochs,test_loss,test_mean_squared_error,time_taken,time_taken_per_epoch,params
4,GANDALFModel,2 M,100,0.062940,0.062940,99.181822,0.991818,"{'task': 'regression', 'head': 'LinearHead', '..."
5,GatedAdditiveTreeEnsembleModel,2 M,100,0.065177,0.065177,1219.995764,12.199958,"{'task': 'regression', 'head': 'LinearHead', '..."
8,TabTransformerModel,277 T,100,0.066654,0.066654,105.431943,1.054319,"{'task': 'regression', 'head': 'LinearHead', '..."
1,CategoryEmbeddingModel,43 T,100,0.067292,0.067292,25.864567,0.258646,"{'task': 'regression', 'head': 'LinearHead', '..."
6,NODEModel,3 M,100,0.068326,0.068326,81.989906,0.819899,"{'task': 'regression', 'head': 'LinearHead', '..."
3,FTTransformerModel,287 T,100,0.068833,0.068833,137.728758,1.377288,"{'task': 'regression', 'head': 'LinearHead', '..."
0,AutoIntModel,27 T,100,0.081260,0.081260,51.428808,0.514288,"{'task': 'regression', 'head': 'LinearHead', '..."
2,DANetModel,1 M,100,0.127868,0.127868,366.805661,3.668057,"{'task': 'regression', 'head': 'LinearHead', '..."
7,TabNetModel,21 T,100,0.127891,0.127891,156.950606,1.569506,"{'task': 'regression', 'head': 'LinearHead', '..."


In [30]:
sweep_df["rmse"] = sweep_df["test_mean_squared_error"]**0.5

In [31]:
sweep_df

,model,# Params,epochs,test_loss,test_mean_squared_error,time_taken,time_taken_per_epoch,params,rmse
4,GANDALFModel,2 M,100,0.062940,0.062940,99.181822,0.991818,"{'task': 'regression', 'head': 'LinearHead', '...",0.250879
5,GatedAdditiveTreeEnsembleModel,2 M,100,0.065177,0.065177,1219.995764,12.199958,"{'task': 'regression', 'head': 'LinearHead', '...",0.255297
8,TabTransformerModel,277 T,100,0.066654,0.066654,105.431943,1.054319,"{'task': 'regression', 'head': 'LinearHead', '...",0.258174
1,CategoryEmbeddingModel,43 T,100,0.067292,0.067292,25.864567,0.258646,"{'task': 'regression', 'head': 'LinearHead', '...",0.259407
6,NODEModel,3 M,100,0.068326,0.068326,81.989906,0.819899,"{'task': 'regression', 'head': 'LinearHead', '...",0.261393
3,FTTransformerModel,287 T,100,0.068833,0.068833,137.728758,1.377288,"{'task': 'regression', 'head': 'LinearHead', '...",0.262361
0,AutoIntModel,27 T,100,0.081260,0.081260,51.428808,0.514288,"{'task': 'regression', 'head': 'LinearHead', '...",0.285061
2,DANetModel,1 M,100,0.127868,0.127868,366.805661,3.668057,"{'task': 'regression', 'head': 'LinearHead', '...",0.357586
7,TabNetModel,21 T,100,0.127891,0.127891,156.950606,1.569506,"{'task': 'regression', 'head': 'LinearHead', '...",0.357619


In [26]:
sweep_df["params"][0]

{'task': 'regression',
 'head': 'LinearHead',
 'head_config': {'layers': ''},
 'embedding_dims': None,
 'embedding_dropout': 0.0,
 'batch_norm_continuous_input': True,
 'learning_rate': 0.001,
 'loss': 'MSELoss',
 'metrics': ['mean_squared_error'],
 'metrics_prob_input': [False],
 'metrics_params': [{}],
 'target_range': None,
 'virtual_batch_size': None,
 'seed': 42,
 '_module_src': 'models.autoint',
 '_model_name': 'AutoIntModel',
 '_backbone_name': 'AutoIntBackbone',
 '_config_name': 'AutoIntConfig',
 'attn_embed_dim': 32,
 'num_heads': 2,
 'num_attn_blocks': 3,
 'attn_dropouts': 0.0,
 'has_residuals': True,
 'embedding_dim': 16,
 'embedding_initialization': 'kaiming_uniform',
 'embedding_bias': True,
 'share_embedding': False,
 'share_embedding_strategy': 'fraction',
 'shared_embedding_fraction': 0.25,
 'deep_layers': False,
 'layers': '128-64-32',
 'activation': 'ReLU',
 'use_batch_norm': False,
 'initialization': 'kaiming',
 'dropout': 0.0,
 'attention_pooling': False}

In [32]:
test_result = best_model.evaluate(test)
test_result

/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/home/nguyenhoa/miniconda3/envs/.henv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.06673388183116913    │
│  test_mean_squared_error  │    0.06673388183116913    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.06673388183116913,
  'test_mean_squared_error': 0.06673388183116913}]

In [34]:
test_result[0]["test_mean_squared_error"]**0.5

0.25832901856192836

In [28]:
best_model

Layer,Type,Params,In sizes,Out sizes
_backbone,GANDALFBackbone,2027778,?,?
_embedding_layer,Embedding1dLayer,2385,?,?
_head,Sequential,239,?,?
loss,MSELoss,0,?,?


In [29]:
# saving model
joblib.dump(best_model, f"{PROJECT_NAME}/final_models/final.joblib")

['run/all/final_models/final.joblib']